In [4]:
import akshare as ak

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="600744", period="daily", start_date="20170301", end_date='20240910', adjust="")
print(stock_zh_a_hist_df)

              日期    股票代码    开盘    收盘    最高    最低     成交量         成交额    振幅  \
0     2017-03-01  600744  5.21  5.17  5.23  5.16   53251  27617637.0  1.34   
1     2017-03-02  600744  5.18  5.12  5.20  5.12   62439  32172517.0  1.55   
2     2017-03-03  600744  5.12  5.11  5.15  5.08   57487  29376171.0  1.37   
3     2017-03-06  600744  5.11  5.19  5.20  5.09   69253  35801047.0  2.15   
4     2017-03-07  600744  5.19  5.22  5.23  5.16   70536  36746817.0  1.35   
...          ...     ...   ...   ...   ...   ...     ...         ...   ...   
1829  2024-09-04  600744  2.82  2.84  2.86  2.82   69470  19719188.0  1.40   
1830  2024-09-05  600744  2.83  2.85  2.86  2.83   66133  18807485.0  1.06   
1831  2024-09-06  600744  2.85  2.82  2.86  2.82   63367  17950979.0  1.40   
1832  2024-09-09  600744  2.81  2.76  2.82  2.75  123931  34320877.0  2.48   
1833  2024-09-10  600744  2.76  2.78  2.78  2.72  104206  28706747.0  2.17   

       涨跌幅   涨跌额   换手率  
0    -0.96 -0.05  0.64  
1    -0.97 -0

In [5]:
price=stock_zh_a_hist_df.loc[:,'收盘']

print(price)

0       5.17
1       5.12
2       5.11
3       5.19
4       5.22
        ... 
1829    2.84
1830    2.85
1831    2.82
1832    2.76
1833    2.78
Name: 收盘, Length: 1834, dtype: float64


In [6]:
import vectorbt as vbt

In [7]:
import vectorbt as vbt
import numpy as np
fast_ma = vbt.MA.run(price, 10)
slow_ma = vbt.MA.run(price, 50)
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=100)
pf.total_profit()

n = np.random.randint(10, 101, size=1000).tolist()
pf = vbt.Portfolio.from_random_signals(price, n=n, init_cash=100, seed=42)



In [9]:
price, _ = price.vbt.range_split(n=3)

# Define hyper-parameter space
# 49 fast x 49 slow x 19 signal
fast_windows, slow_windows, signal_windows = vbt.utils.params.create_param_combs((product, (combinations, np.arange(2, 51, 1), 2), np.arange(2, 21, 1)))

# Run MACD indicator
macd_ind = vbt.MACD.run(
    price,
    fast_window=fast_windows,
    slow_window=slow_windows,
    signal_window=signal_windows
)

# Long when MACD is above zero AND signal
entries = macd_ind.macd_above(0) & macd_ind.macd_above(macd_ind.signal)

# Short when MACD is below zero OR signal
exits = macd_ind.macd_below(0) | macd_ind.macd_below(macd_ind.signal)

# Build portfolio
pf = vbt.Portfolio.from_signals(
    price.vbt.tile(len(fast_windows)), entries, exits, fees=0.001, freq='1D')

# Draw all window combinations as a 3D volume
fig = pf.total_return().vbt.volume(
    x_level='macd_fast_window',
    y_level='macd_slow_window',
    z_level='macd_signal_window',
    slider_level='split_idx',
    trace_kwargs=dict(
        colorbar=dict(
            title='Total return', 
            tickformat='%'
        )
    )
)
fig.show()

NameError: name 'product' is not defined